# Project 3 

#### Jason Kim 
#### Braulio Lopez

### Part 1 

#### Importing the libraries

In [1]:
pip install tensorflow_datasets --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import seaborn as sns
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from pathlib import Path
import keras
from keras.datasets import mnist
import random
import shutil
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

/tmp/ipykernel_4362/952184717.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-04-10 02:31:58.392249: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 02:31:58.440383: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 02:31:58.440438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin c

#### Loading Dataset 

In [3]:
try:
    shutil.rmtree("data_all_modified-cnn-split/train")
    shutil.rmtree("data_all_modified-cnn-split/test")
except:
    pass

In [4]:
Path("data_all_modified-cnn-split/train/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-cnn-split/train/no_damage").mkdir(parents=True, exist_ok=True)

Path("data_all_modified-cnn-split/test/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-cnn-split/test/no_damage").mkdir(parents=True, exist_ok=True)

In [5]:
all_damage_file_paths = os.listdir('data_all_modified/damage')
all_no_damage_file_paths = os.listdir('data_all_modified/no_damage')

In [6]:
train_damage_paths = random.sample(all_damage_file_paths, int(len(all_damage_file_paths)*0.8))
print("train damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in all_damage_file_paths if p not in train_damage_paths]
print("test damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))

train_no_damage_paths = random.sample(all_no_damage_file_paths, int(len(all_no_damage_file_paths)*0.8))
print("train no damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in all_no_damage_file_paths if p not in train_no_damage_paths]
print("test no damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))

train damage image count:  11336
test damage image count:  2834
len of overlap:  0
train no damage image count:  5721
test no damage image count:  1431
len of overlap:  0


In [7]:
# ensure to copy the images to the directories
for p in train_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-cnn-split/train/damage', p) )

for p in test_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-cnn-split/test/damage', p) )

for p in train_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-cnn-split/train/no_damage', p) )

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-cnn-split/test/no_damage', p) )

# check counts:
print("Files in train/damage: ", len(os.listdir("data_all_modified-cnn-split/train/damage")))
print("Files in train/no_damage: ", len(os.listdir("data_all_modified-cnn-split/train/no_damage")))
print("Files in test/damage: ", len(os.listdir("data_all_modified-cnn-split/test/damage")))
print("Files in test/no_damage: ", len(os.listdir("data_all_modified-cnn-split/test/no_damage")))

Files in train/damage:  11336
Files in train/no_damage:  5721
Files in test/damage:  2834
Files in test/no_damage:  1431


#### Data Processing

In [8]:
train_data_dir = 'data_all_modified-cnn-split/train'
batch_size = 32
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.


In [9]:
test_data_dir = 'data_all_modified-cnn-split/test/'

batch_size = 2

# this is what was used in the paper --
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

# approach 1: manually rescale data --
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 4265 files belonging to 2 classes.


In [10]:
### 

In [11]:
from PIL import Image
import os

# Path to the directory containing your images
dataset_dir = 'data_all_modified/no_damage'

image_dimensions = {}

# Loop through each image file in the directory
for filename in os.listdir(dataset_dir):
    if filename.endswith('.jpeg') or filename.endswith('.png'):  # Adjust file extensions as needed
        # Open the image using PIL
        img = Image.open(os.path.join(dataset_dir, filename))
        # Get the dimensions of the image
        width, height = img.size
        # Add dimensions to the dictionary
        if (width, height) in image_dimensions:
            image_dimensions[(width, height)] += 1
        else:
            image_dimensions[(width, height)] = 1

# Print summary of image dimensions
for dimensions, count in image_dimensions.items():
    print(f"{count} images with dimensions {dimensions[0]}x{dimensions[1]}")

7152 images with dimensions 128x128


### Part 2

#### ANN

In [12]:
from tensorflow.keras import models, layers

model_ann = models.Sequential()

# Adjust the input_shape to match the actual image size
model_ann.add(layers.Flatten(input_shape=(128, 128, 3)))  # This line is changed

model_ann.add(layers.Dense(512, activation='relu'))
model_ann.add(layers.Dropout(0.5))  # Optional: Helps prevent overfitting
model_ann.add(layers.Dense(256, activation='relu'))
model_ann.add(layers.Dense(1, activation='sigmoid'))  # For binary classification

model_ann.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

model_ann.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 49152)             0         
                                                                 
 dense (Dense)               (None, 512)               25166336  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 25297921 (96.50 MB)
Trainable params: 25297921 (96.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
history_ann = model_ann.fit(
    train_rescale_ds,
    epochs=1,
    validation_data=val_rescale_ds
)

427/427 [==============================] - 59s 138ms/step - loss: 0.6421 - accuracy: 0.6632 - val_loss: 0.6361 - val_accuracy: 0.6681


In [17]:
test_loss, test_accuracy = model_ann.evaluate(test_rescale_ds, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Test Loss: 0.6383740901947021
Test Accuracy: 0.6644783020019531


In [32]:
model.save('./models/ANN_Model.h5')

#### LeNet-5

In [22]:
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_lenet5.add(layers.Dense(3, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d_6 (Avera  (None, 63, 63, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 16)        880       
                                                                 
 average_pooling2d_7 (Avera  (None, 30, 30, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_4 (Flatten)         (None, 14400)             0         
                                                                 
 dense_12 (Dense)            (None, 120)              

In [23]:
#fit the model from image generator
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=1,
            validation_data=val_rescale_ds
)

427/427 [==============================] - 23s 53ms/step - loss: 0.6125 - accuracy: 0.6880 - val_loss: 0.5574 - val_accuracy: 0.7640


In [25]:
test_loss, test_accuracy = model_lenet5.evaluate(test_rescale_ds, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.56317538022995
Test Accuracy: 0.7650644779205322


In [33]:
model.save('./models/Lenet5_cnn.h5')

#### Alternate-Lenet-5 CNN architecture

In [27]:
from keras.layers import LeakyReLU
from keras.regularizers import l2
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (128,128,3)))
# model.add(LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
#model.add(LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
#model.add(LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
#model.add(LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation = 'relu', kernel_regularizer = l2(1e-4)))
#model.add(LeakyReLU(alpha=0.1))
model.add(layers.Dense(1, activation = 'sigmoid'))
#compile the model with RMSprob with learning rate
from keras import optimizers
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics = ['acc'])

#process the jpeg image
#create an image generator
from keras.preprocessing.image import ImageDataGenerator

#train using data augmentation and dropout
train_datagen = ImageDataGenerator(
                    rescale = 1./255,
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255) #validation data should not be augmented

train_generator = train_datagen.flow_from_directory(
                    'data_all_modified-cnn-split/train', 
                    target_size = (128,128),
                    batch_size = 32,
                    class_mode = 'binary')
validation_generator = test_datagen.flow_from_directory(
                    'data_all_modified-cnn-split/test', 
                    target_size = (128,128),
                    batch_size = 32,
                    class_mode = 'binary')

#fit the model from image generator
history = model.fit_generator(
            train_generator,
            steps_per_epoch=100,
            epochs=1,
            validation_data=validation_generator,
            validation_steps=50)


Found 17057 images belonging to 2 classes.
Found 4265 images belonging to 2 classes.


/tmp/ipykernel_4362/1400200379.py:60: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


100/100 [==============================] - 26s 244ms/step - loss: 0.7610 - acc: 0.6528 - val_loss: 0.7081 - val_acc: 0.6812


In [28]:
test_loss, test_accuracy = model.evaluate(test_rescale_ds, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.7222567200660706
Test Accuracy: 0.6644783020019531


In [34]:
model.save('./models/Alternate_Lenet5.h5')